In [5]:
import surprise

In [6]:
import tensorflow as tf

In [7]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split
import zipfile
from surprise import Reader, Dataset, SVD
from surprise import accuracy
import random
from random import randint
import re
import pandas as pd
import copy

<h4> Read in data

In [183]:
# Read data into an array of strings
def read_data():
    with open('./new_data.data') as f:
        all_lines = f.readlines()

    # Prepare the data to be used in Surprise
    reader = Reader(line_format='user item rating timestamp', sep='\t')
    data = Dataset.load_from_file('./new_data.data', reader=reader)
    
    return all_lines, data

In [135]:
# all_lines, data = read_data()

In [10]:
def create_dataframe(data):
    data = [ x.replace('\t', ', ').replace('\n', '') for x in data ]

    df = pd.DataFrame([sub.split(",") for sub in data])
    df.rename(columns={0:'userID', 1:'movieID', 2:'rating', 3: 'timestamp'}, 
                         inplace=True)
    df = df.drop(columns=['timestamp'])
    return df

In [11]:
def strip_content(data):
    r_unwanted = re.compile("[\n\t\r]")
    return r_unwanted.sub(",", data)

In [133]:
# df_final = create_dataframe(all_lines)

In [134]:
# df_final = df_final.astype('int')

Outdated

In [18]:
# def create_dataframe(data):
#     df_data = pd.DataFrame(data)

#     df_data[0] = df_data[0].apply(strip_content)

#     foo = lambda x: pd.Series([i for i in (x.split(','))])

#     df_final = df_data[0].apply(foo)

#     df_final.rename(columns={0:'userID', 1:'movieID', 2:'rating', 3: 'timestamp'}, 
#                      inplace=True)

#     df_final = df_final.drop(columns=[4])
    
#     return df_final

Grid Search KNN

In [19]:
# sim_options = {
#     "name": ["msd", "cosine"],
#     "min_support": [3, 4, 5],
#     "user_based": [False, True],
# }

# param_grid = {"sim_options": sim_options}

# gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> Grid search for best params </h4>

In [20]:
# param_grid = {
#     "n_epochs": [5, 10],
#     "lr_all": [0.002, 0.005],
#     "reg_all": [0.1, 0.2, 0.4, 0.6]
# }
# gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

# gs.fit(data)

# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

<h4> SVD algo chosen, with best params obtained from GS

In [21]:
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

<h4>Cross validation, 5 folds

In [22]:
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

<h4>Create train and test set, apply predictions on test set

In [23]:
def RMSE_predict_train_test(data):
    # sample random trainset and testset
    # test set is made of 25% of the ratings.
    trainset, testset = train_test_split(data, test_size=.25)

    # We'll use the famous SVD algorithm.
    algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

    # Train the algorithm on the trainset, and predict ratings for the testset
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Then compute RMSE
    return accuracy.rmse(predictions)

<h4> Training Time
   

In [24]:
import timeit

def training_time(data):
    start = timeit.default_timer()

    trainset = data.build_full_trainset()

    algo.fit(trainset)


    stop = timeit.default_timer()

    print('Time: ', stop - start)  

In [25]:
# training_time(data_other)

<h4>Predict scores for all users for all movies. Test the speed of the predictions.

In [26]:
def predict_scores(data):
    # Build Training set. Needed to fit to create model.
    trainset = data.build_full_trainset()
    
    algo.fit(trainset)
    
    # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    # Create empty list to store predictions
    ratings = []
    
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings.append(prediction)
            
    return ratings

<h4> Time to predict all ratings

In [27]:
# import timeit

# start = timeit.default_timer()


# ratings = predict_scores(data_other)
    
# stop = timeit.default_timer()

# print('Time: ', stop - start)  

<h4> Create Dataframe

In [136]:
def predict_scores_dict(data):
    # Build Training set. Needed to fit to create model.
    print("build trainingset")
    trainset = data.build_full_trainset()
    
    print("training algo")
    algo.fit(trainset)
    
    # Get all the user and item IDs
    user_ids = trainset.all_users()
    item_ids = trainset.all_items()
    
    # Create empty list to store predictions
    ratings = {}
    ratings_list = []
    
    print("start prediction")
    # For loop, estimate rating of each user for every movie.
    for user_id in user_ids:
        for item_id in item_ids:
            
            prediction = algo.predict(str(user_id), str(item_id)).est
#             print(prediction)
            ratings['userID'] = int(user_id)
            ratings['movieID'] = int(item_id)
            ratings['rating'] = prediction
            
            ratings_list.append(ratings)
            
            ratings = {}
            
    return ratings_list

In [137]:
# import timeit

# start = timeit.default_timer()

# predicted_scores_dict =  predict_scores_dict(data)
    
# stop = timeit.default_timer()

# print('Time: ', stop - start)  


In [138]:
complete_prediction = pd.DataFrame(predicted_scores_dict)

In [ ]:
# complete_prediction = []
# complete_prediction = pd.DataFrame(complete_pr²ediction)
# for i in range(0, len(predicted_scores_dict), 10000000):
#     print("appending",i, i+10000000)
#     complete_prediction = complete_prediction.append(predicted_scores_dict[i:i+10000000])

<h4> Create extended dataset

In [ ]:
def create_data(data):
    trainset = data.build_full_trainset()

    user_ids = trainset.all_users()
    item_ids = trainset.all_items()

    data_list = []
    # Create new movies (168200 in total)
    for movie in range(item_ids[-1]+1, item_ids[-1]*100):
        # For every movie, there will be 100 users rating the movie
        user_generated = [randint(0, user_ids[-1]) for p in range(0, 100)]
        for user in user_generated:
            # Create a random generated score for the movies
            new_data = str(user)+'\t'+str(movie)+'\t'+str(random.randint(1,5))+'\t'+'NaN\n'

            data_list.append(new_data)
    return data_list

In [ ]:
# data_newest = create_data(data)

Create the new data (as a list)

In [ ]:
# new_data = []

# new_data.extend(all_lines)
# new_data.extend(data_newest)

Check difference in sizes


In [ ]:
# len(data_newest)/len(all_lines)

<h4> Search Function </h4>

In [109]:
def drop_dups(userID, df_final, predicted_scores_df):

    df_user0_predicted = predicted_scores_df[predicted_scores_df['userID'] == userID]
    # df_user0_predicted.sort_values('rating', ascending=False)
    df_user0_predicted.reset_index(drop=True, inplace=True)

    df_user0_original = df_final[df_final['userID'] == userID]
    df_user0_original.reset_index(drop=True, inplace=True)

    dfs_dictionary = {'DF1':df_user0_predicted,'DF2':df_user0_original}
    df3=pd.concat(dfs_dictionary)
    df3=df3.drop_duplicates(subset=['userID', 'movieID'],keep=False)
    
    return df3

In [169]:
def best_movies_for_user(userID, amount_of_movies, df_final, predicted_scores_df):
    ascending_ratings = drop_dups(userID, df_final, predicted_scores_df).sort_values('rating', ascending=False)
    
    return ascending_ratings[0:amount_of_movies]

In [139]:
def insert_into_dataframe(predicted_scores_dict):
    complete_prediction = []
    complete_prediction = pd.DataFrame(complete_prediction)
    for i in range(0, len(predicted_scores_dict), 10000000):
        print("appending",i, i+10000000)
        complete_prediction = complete_prediction.append(predicted_scores_dict[i:i+10000000])
        
    return complete_prediction

In [141]:
def prepare_data():
    all_lines, data = read_data()
    
    print("create df_final")
    df_final = create_dataframe(all_lines).astype('int')
    
    print("predict scores")
    predicted_scores_dict =  predict_scores_dict(data)
    
    df_predicted_scores = insert_into_dataframe(predicted_scores_dict)
#     df_predicted_scores = pd.DataFrame(predicted_scores_dict)
    
    return df_final, df_predicted_scores

In [182]:
# df_final, df_predicted_scores = prepare_data()

In [185]:
df_final2, df_predicted_scores2 = prepare_data()

create df_final
predict scores
build trainingset
training algo
start prediction
appending 0 10000000


In [190]:
import timeit

start = timeit.default_timer()

best_movies = best_movies_for_user(1,10,df_final2, df_predicted_scores2)    
stop = timeit.default_timer()

print('Time: ', stop - start)  


Time:  0.012946999999257969


In [193]:
df_predicted_scores2[df_predicted_scores2['rating']>=5]

,userID,movieID,rating
852786,507,12,5.0
852824,507,50,5.0
852838,507,64,5.0
852888,507,114,5.0
852943,507,169,5.0
852952,507,178,5.0
853092,507,318,5.0
853182,507,408,5.0
853257,507,483,5.0
853377,507,603,5.0
